In [1]:
import os

## 1. 選擇使用的GPU 

In [2]:
# manually select one or several free gpu
os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1'
# use CPU only
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

## 2. 到當前資料夾
- cd C:/Users/Garin/Desktop/學長畢業資料/實驗程式yu/

## 3. 同步程式
### - local -> server
- scp -r -P 2222 ./config garin@140.113.214.40:/home/garin/Documents/depth/
- scp -r -P 2222 ./src garin@140.113.214.40:/home/garin/Documents/depth/

### - server -> local 
- scp -r -P 2222 garin@140.113.214.40:/home/garin/Documents/depth/config .
- scp -r -P 2222 garin@140.113.214.40:/home/garin/Documents/depth/src .

## 4. 建dataset

In [ ]:
%run data_preparation/kitti_data_prepare.py \
--dataset_dir=/home/mjchiu/Documents/darknet-depth/dataset/KITTI/image/ \
--dataset_name=kitti_raw_eigen \
--dump_root=/home/garin/Documents/depth/datasets/kitti_3frames_256_832/ \
--seq_length=3 \
--img_height=256 \
--img_width=832 \
--num_threads=16 \
--remove_static

## 5. Train flow 

In [ ]:
%run ./main.py -c ../config/flow3.ini -t train_flow --cont_model=../results/KITTI_RAW_256_832_UnDepthflow_flow_pwc_b8_3frames/checkpoints/kitti_3frames/model-130768

[!] Loading the model for 3 frames...
[Info] Building flow network ...
[Info] img_height: 256 img_width 832


/home/garin/anaconda3/envs/python3.5_tf1.9/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[Info] Building flow network ...
[Info] img_height: 256 img_width 832
[Info] Model size: 5.11574M
[Info] Continue training. Restoreing: ../results/KITTI_RAW_256_832_UnDepthflow_flow_pwc_b8_3frames/checkpoints/kitti_3frames/model-130768
INFO:tensorflow:Restoring parameters from ../results/KITTI_RAW_256_832_UnDepthflow_flow_pwc_b8_3frames/checkpoints/kitti_3frames/model-130768


In [ ]:
%run ./main.py -c ../config/flow3.ini -t train_flow

## 6. Train depth & pose 

In [ ]:
%run ./main.py -c ../config/dp3.ini -t train_dp \
--restore_flow_model=../results/KITTI_RAW_256_832_UnDepthflow_flow_pwc_b8_3frames/checkpoints/kitti_3frames/model-130768

#### continue training depth and pose 

In [ ]:
%run ./main.py -c ../config/dp3.ini -t train_dp \
--cont_model=../results/KITTI_RAW_128_416_UnDepthflow_dp_b4_resnet50_3frames/checkpoints/kitti_3frames/model-342007 \
--restore_flow_model=../results/KITTI_RAW_256_832_UnDepthflow_flow_pwc_b8_3frames/checkpoints/kitti_3frames/model-130768

## 7. Test

In [1]:
%run ./main.py -c ../config/test_dp_kitti.ini -t kitti_eval \
--restore_dp_model=../results/yu_result/model-110001
# --restore_dp_model=../results/KITTI_RAW_128_416_UnDepthflow_dp_b4_resnet50_3frames/checkpoints/kitti_3frames/model-342007


[Info] Evaluate kitti depth...
[Info] Reading datalist from: /home/garin/Documents/depth/src/kitti_eval/kitti/test_files_eigen.txt
[Info] Loading images from: /home/waterman/dataset/KITTI
[Info] Reshaing image to size: (256, 832)
[Info] Restoring model: ../results/yu_result/model-110001
[Info] Data number: 697
[Info] FPS: 35.884
[Info] Saving to ../results/kitti/test_kitti.npy


In [ ]:
%run kitti_eval/eval_depth.py --split=eigen --kitti_dir=/home/waterman/dataset/KITTI/ \
--pred_file=../results/kitti/test_kitti.npy \
--depth_results=../results/kitti_depths

#### 將結果傳回本地端 

scp -r -P 2222 garin@140.113.214.40:/home/garin/Documents/depth/results/kitti_depths/ C:/Users/Garin/Desktop/學長畢業資料/實驗程式yu/result